# How to Perform N Times Faster Outlier Detection in UMAP (Python) on Massive Datasets
## SUBTITLE TODO
![](images/pixabay.jpg)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://pixabay.com/users/publicdomainpictures-14/?utm_source=link-attribution&utm_medium=referral&utm_campaign=image&utm_content=2205'>PublicDomainPictures</a>
        on 
        <a href='https://pixabay.com/?utm_source=link-attribution&utm_medium=referral&utm_campaign=image&utm_content=2205'></a>
    </strong>
</figcaption>